# Construção da base a partir do JSON

________
_______
_______

In [ ]:
import numpy as np
import pandas as pd
import json
import re

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = "{:.2f}".format

In [ ]:
# função de processamento de texto

from unicodedata import normalize

def tira_acento(x):
    
    return normalize('NFKD', x).encode('ASCII','ignore').decode('ASCII')

In [ ]:
# função de exibição 

def show_all(df):
    
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_columns', None)
    
    display(df)
    
    pd.reset_option('display.max_colwidth')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.max_columns')
    

______

### 1) Leitura dos dados

In [ ]:
from os import listdir
from os.path import isfile, join
import time

path = "../data/"

print("\nLista de arquivos disponíveis para a leitura:\n")

file_list = [str(i) + " - " + f for i, f in enumerate(listdir(path)) if isfile(join(path, f))]

print(*file_list, sep="\n")

time.sleep(0.5)

num = int(input("\nDigite o número correspondente ao arquivo desejado: (0-" + str(len(file_list)) + "): "))

file_path = path + file_list[num].split(" - ")[-1]

file_path

In [ ]:
# lê o arquivo e cria lista com json de cada carro

with open(file_path) as f:
    
    json_list = json.load(f)

In [ ]:
# identificando quais chaves têm listas como valores
# nestes casos, o json_normaliza falha, por isso é importante identificarmos
# antes pra fazer o flatenning

list_values = []
for item in json_list:
    
    for key in item.keys():

        if type(item[key]) == list:

            # print(key, item[key])
            
            if key not in list_values:
                
                list_values.append(key)
                
if len(list_values) > 0:
    
    print("\nHá jsons com chaves cujos valores são listas!!")
    print("Estas chaves são:")
    print(list_values)
          
    print("\nVamos corrigir isso agora!")
    
    # fazendo o flattening dos valores que são listas

    for item in json_list:

        for key in list_values:

            try:
                for i in range(len(item[key])):

                    item[key + "_" + str(i+1)] = item[key][i]

                item.pop(key)

            except:

                pass
    
    print("\nTudo corrigido! Podemos seguir com a construção do df!")
    
# se não tiver nenhuma chave com essa característíca, ótimo!
else:
    
    print("\nNão há problemas com json algum, tudo certo!")
    print("\nPodemos seguir diretamente com a construlção do df!")

In [ ]:
# montando o df

df = pd.json_normalize(json_list)

In [ ]:
df.shape

In [ ]:
df.columns.tolist()

In [ ]:
show_all(df.head(3))

In [ ]:
df.info()

In [ ]:
print("\nColunas com dados missing em respectiva proporção:")

# colunas com dados missing, apenas
# contagem já normalizada
missing = df.isnull().sum()[df.isnull().sum().apply(lambda x: True if x!=0 else False)]/df.shape[0]

# print formatado
missing.apply(lambda x: "{:.2f}%".format(x*100))

_______

### 2) Pré-processamento

In [ ]:
# caso queira evitar que alguma coluna seja dropada

keep = []

In [ ]:
# vamos dropar as colunas que tenham mais que 50% de NaNs

threshold_drop = 0.5

print("\nColunas com proporção de NaNs maior que o threshold de {:.0f}%:\n".format(threshold_drop*100))

display(missing[missing>threshold_drop].apply(lambda x: "{:.2f}%".format(x*100)))

drop_nans = missing[missing>threshold_drop].index.tolist()

# evitando que colunas em "keep" sejam dropadas
drop_nans = [x for x in drop_nans if x not in keep]

# dropando!
df = df.drop(columns=drop_nans)

print("\nColunas acima dropadas com sucesso!")

In [ ]:
# for item in df.columns.tolist():
    
#     print(f'"{item}" : "{item}",')

In [ ]:
# renomeando colunas

rename_dic = {  "LongComment" : "comentario",
                "FipePercent" : "fipe_perc",
                "unico_dono" : "unico_dono",
                "ipva_pago" : "ipva_pago",
                "aceita_troca" : "aceita_troca",
                "financiado" : "financiado",
                "licenciado" : "licenciado",
                "contains_leilao" : "contains_leilao",
                "Specification.Title" : "titulo",
                "Specification.Make.Value" : "marca",
                "Specification.Model.Value" : "modelo",
                "Specification.Version.Value" : "versao",
                "Specification.YearFabrication" : "ano",
                "Specification.YearModel" : "ano_modelo",
                "Specification.Odometer" : "km",
                "Specification.Transmission" : "transmissao",
                "Specification.NumberPorts" : "num_portas",
                "Specification.BodyType" : "categoria",
                "Specification.Armored" : "blindado",
                "Specification.Color.Primary" : "cor",
                "Seller.SellerType" : "vendedor_tipo",
                "Seller.City" : "vendedor_cidade",
                "Seller.State" : "vendedor_estado",
                "Seller.DealerScore" : "score_vendedor",
                "Seller.CarDelivery" : "car_delivery",
                "Seller.TrocaComTroco" : "troca_troco",
                "Seller.ExceededPlan" : "exceeded_plan",
                "Prices.Price" : "preco",
                "Prices.SearchPrice" : "preco_busca"}

df = df.rename(columns=rename_dic)

# determinando a orderm

cols_select = [ # atributos gerais
               'titulo', 'marca', 'modelo', 'versao', 
               'ano', 'ano_modelo', 
               'km', 'transmissao', 'num_portas',
               'categoria', 'blindado', 'cor',
                # preço
               'preco', 'preco_busca', 'fipe_perc', 
                # flags binarias
               'unico_dono', 'ipva_pago', 'aceita_troca', 'financiado', 'licenciado', 'contains_leilao',
                # info vendedor
               'vendedor_tipo', 'vendedor_cidade', 'vendedor_estado', 'score_vendedor',
                # extras
               'car_delivery', 'troca_troco', 'comentario',
                # 'exceeded_plan'
                ]

df = df[cols_select]

In [ ]:
show_all(df.head(3))

In [ ]:
# isso vai auxiliar na correção dos dtypes

df.info()

In [ ]:
# ajustando dtypes

df["ano"] = df["ano"].astype(int)

df["ano_modelo"] = df["ano_modelo"].astype(int)

df["num_portas"] = df["num_portas"].astype(int)

In [ ]:
df.info()

In [ ]:
# avaliando ststs descritivas procurando outliers absurdos

df.describe()

_______

### 3) Feature engineering

In [ ]:
# pegando cilindradas (1.0, 1.6, etc) a partir da coluna "versão" (proxy)

# obs: "Todo carro é movido por um motor à combustão, a diferença de um motor 
# para o outro é quantidade de cilindradas. 
# Quando falamos que um carro é 2.0, isso quer dizer que ele tem 2000 cilindradas. 
# O mesmo acontece com o 1.0 (1000 cc), 1.4 (1400), 1.6 (1.600) e 1.8 (1800)." 

df["cilindrada"] = df["versao"].apply(lambda x: 
                                       re.findall("\d+\.\d+", x)).apply(lambda x: 
                                                                        x[0] if len(x) == 1 else np.nan)

###############################################################

df["cilindrada"].value_counts(dropna=False)

In [ ]:
# pegando o tipo de combustível do carro a partir da coluna "versão" (proxy)

def tipo_combustivel(x):

    ans = "OUTRO"
    
    for s in ["GASOLINA", "ALCOOL","DIESEL", "FLEX", "HIBRIDO", "HYBRID"]: 

        if s in tira_acento(x).upper(): 

            ans = s
    
    return ans
        
df["combustivel"] = df["versao"].apply(tipo_combustivel)

###############################################################

df["combustivel"].value_counts()

_______

### 4) Análise preliminar - marcas e modelos

In [ ]:
# dic de dfs pra cada marca
# estrutura: {"marca" : df_com_marca_filtrada}

marcas = df["marca"].unique().tolist()

df_marcas = {marca: df[df["marca"] == marca] for marca in marcas}

In [ ]:
# dic com a seguinte estrutura: {"marca": {"modelo_i" : df_modelo_i.describe()}}

# essa célula pode demorar um pouquinho...

modelos_describe = {marca : 
                           {modelo: df_marcas[marca].loc[df_marcas[marca]["modelo"] == modelo].describe() 
                            for modelo in df_marcas[marca]["modelo"].unique().tolist()}
                    for marca in marcas}

In [ ]:
print("\nMarcas disponíveis:\n")

print(list(modelos_describe.keys()))

In [ ]:
# pegando modelos de determinada marca 

print(list(modelos_describe["HYUNDAI"].keys()))

In [ ]:
# pegando stats de determinado modelo

modelos_describe["HYUNDAI"]['HB20']

In [ ]:
# groupby - {marca: {modelo_i :  groupby_i(ano, cilindrada, combustivel)}}
# esse tbm demora um pouco

marca_modelo_ano = {marca : 
                        {modelo: 
                                 df_marcas[marca][df_marcas[marca]["modelo"] == modelo].groupby(["ano",
                                                                                                "cilindrada",
                                                                                                "combustivel"])[["km",
                                                                                                                 "preco",
                                                                                                                 "fipe_perc"]].agg(["count", 
                                                                                                                                    "mean",
                                                                                                                                    "min",
                                                                                                                                    "max",
                                                                                                                                    np.median]) 
                        for modelo in df_marcas[marca]["modelo"].unique().tolist()}
                    for marca in marcas}

In [ ]:
marca_modelo_ano["HYUNDAI"]['HB20']

______

__Vamos pegar as N marcas mais comuns (em termos de count de carros)__

Destas, vamos rodar o DBSCAN pra cada modelo. Com isso, encontraremos outliers.

In [ ]:
# top N

N = 10

topN = df["marca"].value_counts().iloc[:N].index

topN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

topN_dict = {}

for marca in topN:
    
    topN_dict[marca] = {}
    
    print("\n#############################################################")
    print("#############################################################")
    print("~~~~~~\tMarca:", marca)
    print("#############################################################")
    print("#############################################################\n")
    
    aux = df_marcas[marca]
    
    for modelo in aux["modelo"].unique():

        aux2 = aux[aux["modelo"] == modelo].copy()
        
        # apenas se tiver mais que um exemplar do modelo na base
        if aux2.shape[0] > 1:
            
            print("\nMODELO:", modelo)
            
            #################################################################################
#             # visualizando preço vs km com hue de ano
#             f = sns.scatterplot(data=aux2, y="preco", x="km", 
#                                 hue="ano", palette="rainbow")

#             ax = plt.gca()
#             ax.set_title(modelo)

#             plt.show()
            #################################################################################
            
            # clustering
            
            # selecionando features - apenas numeric & bool
            X = aux2.select_dtypes(include=[np.number, np.bool]).copy()
            
            # selecionando apenas algumas cols - sempre inclua preco!
            cols_features = ["preco", "km"]
            X = aux2[cols_features]
            
            # preenchendo NaNs
            cols_with_nan = X.isnull().any(axis=0)[X.isnull().any(axis=0) == True].index

            for col in cols_with_nan:

                # definindo os atributos pro cruzamento
                cols = ['transmissao', 'categoria', 'blindado', 'cilindrada', 'combustivel']

                # fazendo o agrupamento
                media_col = aux2.groupby(cols)[[col]].mean().reset_index()

                # sub-base com valores missing de "col", e apenas as colunas de cruzamento
                X_col_missing = aux2.loc[X[col].isnull()][cols]

                # fazendo o cruzamento
                valores_a_preencher = X_col_missing.merge(media_col, 
                                                          on=cols, 
                                                          how="left").set_index(X_col_missing.index)[col]

                # preenchendo vazio com as médias
                X.loc[df[col].isnull(), col] = valores_a_preencher
            
            # normalização
            X = pd.DataFrame(StandardScaler().fit_transform(X), columns = X.columns, index= X.index)
            
            #################################################################################
#             # visualizando a normalização
#             f = sns.scatterplot(data=X, y="preco", x="km", palette="rainbow")

#             ax = plt.gca()
#             ax.set_title(modelo)

#             plt.show()
            #################################################################################
    
            # modelagem
            db = DBSCAN(eps=0.5, min_samples=2, metric="euclidean").fit(X)
            
            # cluster labels
            aux2["cluster"] = db.labels_
            
            #################################################################################
            # visualizando preço vs km com hue de cluster
            f = sns.scatterplot(data=aux2, y="preco", x="km", 
                                hue="cluster", palette="rainbow")

            ax = plt.gca()
            ax.set_title(modelo)

            plt.show()
            #################################################################################
            
            # atualiza o dict com o aux2
            topN_dict[marca][modelo] = aux2

            
        else:
            
            print("\nHá apenas um exemplar do modelo:", modelo)
            print("Portanto, não é possível estudá-lo estatisticamente.")
            
        
#         display(aux2)
        
    
# del aux, aux2

In [ ]:
topN_dict.keys()

In [ ]:
topN_dict['HYUNDAI'].keys()

In [ ]:
marca = 'HYUNDAI'
modelo = "HB20"

aux = topN_dict[marca][modelo]

display(aux[aux["cluster"] == -1][["marca", "modelo", "ano", "km", "preco"]])

f = sns.scatterplot(data=aux, y="preco", x="km", 
                    hue="cluster", palette="rainbow")

ax = plt.gca()
ax.set_title(marca + " " + modelo)

plt.show()

In [ ]:
# fipe_perc por marca - visão de valorização por marca

df.groupby("marca")[["fipe_perc"]].agg(["mean", "median", "count"])

In [ ]:
# fipe_perc por marca e modelo - visão de valorização por marca

df.groupby(["marca", "modelo"])[["fipe_perc"]].agg(["mean", "median", "count"])

In [ ]:
aux = df_marcas["HYUNDAI"][df_marcas["HYUNDAI"]["modelo"] == "HB20"]

aux = aux[aux["km"] < 5e5]

plt.figure(figsize=(16, 8))

sns.jointplot(data=aux, y="preco", x="km", hue="ano", palette="rainbow")

plt.show()

In [ ]:
for ano in aux["ano"].unique():
    
    aux2 = aux[aux["ano"] == ano]
    
    sns.jointplot(data=aux2, y="preco", x="km", hue="ano", palette="rainbow")
    
    plt.show()

In [ ]:
aux[(aux["km"] < 10000) & (aux["preco"] <40000)] 

______

### 4) Análise por faixas

In [ ]:
# para auxiliar a definir as faixas...

df[["preco", "km", "ano"]].describe()

In [ ]:
for col in ["preco", "km", "ano"]:
    
    plt.figure(figsize=(12, 6))

    plt.title("Distribuição - coluna " + col , size=18)

    sns.histplot(df[col], kde=True, alpha=0.2)

    # plotando média
    plt.axvline(x=df[col].mean(), color="red", label="média")

    # plotando a mediana
    plt.axvline(x=df[col].median(), color="green", label="mediana")

    # plotando a(s) moda(s)
    for i in range(df[col].mode().shape[0]):

        plt.axvline(x=df[col].mode()[i], color="yellow", label="moda")

    plt.legend()
    plt.show()

In [ ]:
col = "preco"

plt.figure(figsize=(12, 6))

plt.title("Distribuição de preços" , size=18)

sns.histplot(df[col], kde=True, alpha=0.2)

# plotando média
plt.axvline(x=df[col].mean(), color="red", label="média")

# plotando a mediana
plt.axvline(x=df[col].median(), color="green", label="mediana")

# plotando a(s) moda(s)
for i in range(df[col].mode().shape[0]):

    plt.axvline(x=df[col].mode()[i], color="yellow", label="moda")

plt.legend()
plt.show()

############################################

plt.figure(figsize=(12, 6))

plt.title("Distribuição de log10-preços" , size=18)

sns.histplot(np.log10(df[col]), kde=True, alpha=0.2)

# plotando média
plt.axvline(x=np.log10(df[col]).mean(), color="red", label="média")

# plotando a mediana
plt.axvline(x=np.log10(df[col]).median(), color="green", label="mediana")

# plotando a(s) moda(s)
for i in range(np.log10(df[col]).mode().shape[0]):

    plt.axvline(x=np.log10(df[col]).mode()[i], color="yellow", label="moda")

plt.legend()
plt.show()

_________

## 5) EDO

Marca, modelo, ano

elimina outliers -- 3, 4, 5 std. km

ipva pago, revisão: importante

aí, olha pra tabela fipe



In [ ]:
df[df["ano"] == 2012]

In [ ]:
df[(df["preco"] < 80e3) & 
   (~df["marca"].isin(["RENAULT", "PEUGEOT", "CITROËN"])) &
   (df["ano"] > 2013)].sort_values("fipe_perc").to_excel("query_edo1.xlsx")

____
____
____
____
____
____
____
____
____
____
____
____
____
____
____
____

In [ ]:
assert(False)

In [ ]:
df["ano_fabricação"] = df["ano_fabricação"].apply(lambda x: int(x))

# claramente um outlier...
df = df[df["preço"] < 1e8]

sns.jointplot(data=df, x="ano_fabricação", y="preço")

In [ ]:
sns.jointplot(data=df, x="km", y="preço")

In [ ]:
plt.figure(figsize=(15, 12))

sns.jointplot(data=df[df["marca"].isin(["CHEVROLET", "VOLKSWAGEN"])], x="km", y="preço", hue="marca")

plt.show()

In [ ]:
df[df["preço"] != df["preço_busca"]]

In [ ]:
df.groupby(["marca", "modelo"])[["modelo"]].count()

In [ ]:
aux = df.groupby(["marca", "modelo", "ano_fabricação"])[["ano_fabricação"]].count()

In [ ]:
aux.loc[['HONDA']]

In [ ]:
.jsonimport requests 

resposta = requests.get("https://fipeapi.appspot.com/api/1/carros/veiculos/21.json")

In [ ]:
resposta.json()